# Exemplos de rag com chroma

## OpenAI

In [1]:
import os
import chromadb
from langchain.document_loaders import TextLoader, PyMuPDFLoader, PythonLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings  # ou outra implementação
from langchain_chroma import Chroma
from uuid import uuid4

# Diretório do projeto Django
PROJECT_DIR = "./django-ecommerce"
# Diretório para o banco de dados ChromaDB
CHROMA_DB_DIR = "./chroma_db_openai"

# Inicializa o modelo de embeddings
embeddings = OpenAIEmbeddings()  # Substitua por outro modelo, se necessário

# Inicializa ChromaDB
chroma_client = chromadb.PersistentClient(path=CHROMA_DB_DIR)
collection = chroma_client.get_or_create_collection(name="django_project_docs")

# Configura a fragmentação de texto
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# Tipos de arquivos suportados e seus loaders
LOADERS = {
    ".py": PythonLoader,
    ".txt": TextLoader,
    ".md": TextLoader,
    ".pdf": PyMuPDFLoader,
    ".html": TextLoader,
    ".js": TextLoader,
}

def ingest_files():
    """Percorre o diretório do projeto Django, carrega e injeta arquivos no ChromaDB."""
    for root, _, files in os.walk(PROJECT_DIR):
        for file in files:
            file_path = os.path.join(root, file)
            ext = os.path.splitext(file)[-1].lower()
            
            if ext in LOADERS:
                try:
                    loader = LOADERS[ext](file_path)
                    documents = loader.load()
                    
                    for doc in documents:
                        chunks = text_splitter.split_text(doc.page_content)
                        metadados = {"source": file_path}
                        
                        for chunk in chunks:
                            embedding = embeddings.embed_query(chunk)
                            collection.add(documents=[chunk], metadatas=[metadados], ids=[str(uuid4())], embeddings=[embedding])
                            
                    print(f"Ingestão concluída: {file_path}")
                except Exception as e:
                    print(f"Erro ao processar {file_path}: {e}")

if __name__ == "__main__":
    ingest_files()
    print("Ingestão finalizada!")


/tmp/ipykernel_9763/1068382742.py:15: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()  # Substitua por outro modelo, se necessário


Ingestão concluída: ./django-ecommerce/README.md
Ingestão concluída: ./django-ecommerce/manage.py
Ingestão concluída: ./django-ecommerce/static/admin/css/vendor/select2/LICENSE-SELECT2.md
Ingestão concluída: ./django-ecommerce/static/admin/fonts/README.txt
Ingestão concluída: ./django-ecommerce/static/admin/fonts/LICENSE.txt
Ingestão concluída: ./django-ecommerce/static/admin/js/autocomplete.js
Ingestão concluída: ./django-ecommerce/static/admin/js/actions.min.js
Ingestão concluída: ./django-ecommerce/static/admin/js/SelectFilter2.js
Ingestão concluída: ./django-ecommerce/static/admin/js/cancel.js
Ingestão concluída: ./django-ecommerce/static/admin/js/inlines.min.js
Ingestão concluída: ./django-ecommerce/static/admin/js/actions.js
Ingestão concluída: ./django-ecommerce/static/admin/js/prepopulate_init.js
Ingestão concluída: ./django-ecommerce/static/admin/js/urlify.js
Ingestão concluída: ./django-ecommerce/static/admin/js/collapse.min.js
Ingestão concluída: ./django-ecommerce/static/ad

KeyboardInterrupt: 

In [ ]:
vector_store = Chroma(
    collection_name="django_project_docs",
    embedding_function=embeddings,
    persist_directory="./chroma_db_openai",  # Where to save data locally, remove if not necessary
)

In [ ]:
results = vector_store.similarity_search(
    "",
    k=5,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

## Google

In [2]:
import os
# from langchain.document_loaders import DirectoryLoader, UnstructuredPDFLoader, TextLoader
from langchain_community.document_loaders import DirectoryLoader, UnstructuredPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma


from langchain_openai import ChatOpenAI

# Configurações
DATA_DIR = "./django-ecommerce"  # Substitua pelo caminho do seu projeto
CHROMA_DIR = "./chroma_db_google"

# Função para carregar documentos de um diretório
def carregar_documentos(diretorio):
    loaders = [
        lambda path: DirectoryLoader(path, glob="**/*.pdf", loader_cls=UnstructuredPDFLoader),
        lambda path: DirectoryLoader(path, glob="**/*.txt", loader_cls=TextLoader),
        lambda path: DirectoryLoader(path, glob="**/*.py", loader_cls=TextLoader),  # Exemplo para arquivos Python
        lambda path: DirectoryLoader(path, glob="**/*.html", loader_cls=TextLoader), # Exemplo para arquivos HTML
        lambda path: DirectoryLoader(path, glob="**/*.md", loader_cls=TextLoader),  # Exemplo para arquivos Markdown
        lambda path: DirectoryLoader(path, glob="**/*.js", loader_cls=TextLoader),  # Exemplo para arquivos JavaScript
        # Adicione outros loaders conforme necessário para os tipos de arquivo do seu projeto
    ]

    documentos = []
    for loader in loaders:
        try:
            documentos.extend(loader(diretorio).load())
        except Exception as e:
            print(f"Erro ao carregar documentos de {diretorio}: {e}")
    return documentos

# Função para dividir documentos em chunks
def dividir_documentos(documentos):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documentos)
    return chunks

# Função para criar embeddings e armazenar no Chroma
def criar_vetorstore(chunks, persist_directory):
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(chunks, embeddings, persist_directory=persist_directory)
    return vectorstore

# Pipeline principal
def pipeline_rag(diretorio, persist_directory):
    documentos = carregar_documentos(diretorio)
    chunks = dividir_documentos(documentos)
    vetorstore = criar_vetorstore(chunks, persist_directory)
    return vetorstore

# Execução da pipeline
vetorstore = pipeline_rag(DATA_DIR, CHROMA_DIR)

print("Pipeline RAG concluída. Banco de dados vetorial Chroma criado em:", CHROMA_DIR)

Pipeline RAG concluída. Banco de dados vetorial Chroma criado em: ./chroma_db_google
